<a href="https://colab.research.google.com/github/johnzelson/local-nonprofit-colab/blob/main/S6_Build_np_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview
---
Step Six (S6 Build np_local)

This notebook combines previous processing into one tidy pile (dataframe) of nonprofit data for analysis and streamlit app.

Adds cluster indicator for scenario where nonprofits are colocated or very close, in order to map without clutter.

Merges: local NPs (selection from BMF), Latest Taxes, Geocode, and Websearch results.  Then congress api.




# Tech Notes

Data from previous processing combined

|  In   |  Description |
|------- | --------------------- |
|np_cortland_p_df | DF of NPs to analyze
|IRS_latest_df    | Latest Tax Return for NP
|cort_geo_df      | Geocoded Info for NP
|web_srch_df      | Web Search Info




| Out   | Description    |
|------- | --------------------- |
| np_local_df  | df that combines earlier processing to use analysis/web app (streamlit) |


# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import requests
import pprint
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import folium
from google.colab import userdata

pd.set_option('display.max_columns', 100);
pd.set_option('display.max_rows', 100);


In [ ]:
#TODO: create a configuration section -- not implemented

# base folder for retrieving raw data
data_dir = '/content/drive/My Drive/irs_data/'

# folder for writing processed data
proc_dir = '/content/drive/My Drive/IRS_processed/'

# folder for code lookups
lu_dir = '/content/drive/My Drive/irs_data/IRS_Code_Lookups/'



# Load and merge dataframes

In [ ]:
# @title Load Incremental Processing and Combine

# the local area is np_cortland_p_df

#TODO: use proc_dir configuration in def or setup cell
proc_dir = '/content/drive/My Drive/IRS_processed/'

#TODO:  Check dtypes
dtype = {"CLASSIFICATION": str,
         "EIN" : str,
         "ACTIVITY" : str,
         "AFFILIATION" : str,
         "ORGANIZATION" : str,
         "FOUNDATION" : str,
         "NTEE_CD" : str,
         "RULING" : str,
         "ZIP" : str,
         "TAX_PERIOD" : str,
         "GROUP" : str
         }

np_cortland_p_df = pd.read_csv('/content/drive/My Drive/IRS_processed/np_cortland_p_df.csv', dtype=dtype)

dtype = {"EIN" : str,
         "ZIPCd" : str
         }
IRS_latest_df = pd.read_csv('/content/drive/My Drive/IRS_processed/irs_latest_df.csv', dtype=dtype)

cort_geo_df = pd.read_csv('/content/drive/My Drive/IRS_processed/cortland_geo_df.csv')

web_srch_df = pd.read_csv('/content/drive/My Drive/IRS_processed/web_srch_df.csv')

# 39sec - 1m 3s


In [ ]:
# @title make curated all_ny... df
#TODO: save a curated version of all NY nonprofits for
#      the streamlit of that gives window into dataset
#      right now, ny_np_p_df is just the basic bmf without tax data
#      It wouldnot include the geocoding or web search results
#      since geoding in only happening to local area


dtype = {"EIN" : str,
         "ZIPCd" : str
         }
ny_np_p_df = pd.read_csv('/content/drive/My Drive/IRS_processed/np_ny_p_df.csv', dtype=dtype)

tmp_all_ny_np_df = ny_np_p_df.merge(IRS_latest_df, on='EIN', how='left')

# curious to see how big it is
tmp_all_ny_np_df.to_csv(proc_dir + 'tmp_all_ny_np_df.csv', index=False)
# 250M

display(tmp_all_ny_np_df)
all_ny_np__df.info(verbose=True)

# get list of columns to decide which to curate for web app
# github limit of 100M
column_names = tmp_all_ny_np__df.columns
column_names_list = list(column_names)
for cn in column_names_list:
  print ("'" + cn +", ")

flds = ['p_org_id',
    'EIN',
'NAME',
'STREET',
'CITY',
'RULING',

'GROUP',
'SUBSECTION',
'AFFILIATION',
'CLASSIFICATION',
'RULING',
'DEDUCTIBILITY',
'FOUNDATION',
'ACTIVITY',
'ORGANIZATION',
'ASSET_AMT',
'INCOME_AMT',
'REVENUE_AMT',
'ASSET_CD',
'INCOME_CD',

'ntee_cat',
'act1_lu',
'act2_lu',
'act3_lu',
'RULE_DT',
'aff_lu',
'org_lu',
'found_lu',
'deduct_lu',
'zipcode',
'major_city',
'uszip_county',
'radius_in_miles',
'area_code_list',
'population',
'population_density',
'housing_units',
'occupied_housing_units',
'median_home_value',
'median_household_income',
'ReturnTypeCd',
'PhoneNum',
'WebsiteAddressTxt',
'ProgramSrvcAccomplishmentGrp',
'Pgm_Serv_Accomps',
'Total_Revenue',
'Mission',
'ActivityOrMissionDesc',
'TotalEmployeeCnt',
'MissionDesc',
'CYTotalRevenueAmt',
        ]
display(tmp_all_ny_np_df[flds])

all_ny_np_df = tmp_all_ny_np_df[flds].copy()
#display(all_ny_np_df)

all_ny_np_df.to_csv(proc_dir + 'all_ny_np_df.csv', index=False)
# 85M, so it is worth it to prune fields

all_ny_np_df.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119765 entries, 0 to 119764
Data columns (total 51 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   p_org_id                      119765 non-null  int64  
 1   EIN                           119765 non-null  object 
 2   NAME                          119765 non-null  object 
 3   STREET                        119765 non-null  object 
 4   CITY                          119765 non-null  object 
 5   RULING                        119765 non-null  int64  
 6   GROUP                         119765 non-null  int64  
 7   SUBSECTION                    119765 non-null  int64  
 8   AFFILIATION                   119765 non-null  int64  
 9   CLASSIFICATION                119765 non-null  int64  
 10  RULING                        119765 non-null  int64  
 11  DEDUCTIBILITY                 119765 non-null  int64  
 12  FOUNDATION                    119765 non-nul

In [ ]:
# review as needed

np_cortland_p_df

display(IRS_latest_df)

np_cortland_p_df.info(verbose=True)

IRS_latest_df.info()

cort_geo_df.info()

web_srch_df


,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,p_org_id,ntee_cat,ntee_define,class_cd_1,class_cd_2,class_cd_3,class_cd_4,activity_cd_1,activity_cd_2,activity_cd_3,act1_lu,act2_lu,act3_lu,RULE_DT,aff_lu,org_lu,found_lu,deduct_lu,zipcode,zipcode_type,major_city,post_office_city,common_city_list,uszip_county,state,radius_in_miles,area_code_list,population,population_density,land_area_in_sqmi,water_area_in_sqmi,housing_units,occupied_housing_units,median_home_value,median_household_income,bounds_west,bounds_east,bounds_north,bounds_south
0,010939780,FREE & ACCEPTED MASONS OF NEW YORK,% JASON C TRABUCCO,15 EXCELSIOR ST,CORTLAND,NY,13045-2301,0265,10,9,1000,194012,2,00,279036000,5,1,202312,0,0,2,0,12,0.0,0.0,0.0,NaN,438 MARATHON LODGE,449,no_NTEE,no_NTEE,1,0,0,0,279,36,0,Other services or benefits to members or emplo...,Fraternity or sorority; Note: Key club (use 323),na,1940-12-01,(9) Subordinate (in Group),Association,code_not_found,Contributions not deductible.,13045,STANDARD,Cortland,"Cortland, NY",['Cortland'],Cortland County,NY,13.0,607,30112.0,283.0,106.58,0.34,12112.0,11217.0,108400.0,43985.0,-76.323463,-76.06199,42.732419,42.469917
1,030469943,DISABLED AMERICAN VETERANS AUXILIARY INC,% CAROL WHITMARSH,14813 STATE ROUTE 90,CORTLAND,NY,13045-0000,2473,4,9,1000,197310,1,00,908921000,5,1,202306,0,0,2,0,6,0.0,0.0,0.0,NaN,153 CORTLAND MEMORIAL,949,no_NTEE,no_NTEE,1,0,0,0,908,921,0,Patriotic activities,Loans or credit reporting,na,1973-10-01,(9) Subordinate (in Group),Association,code_not_found,Contributions deductible.,13045,STANDARD,Cortland,"Cortland, NY",['Cortland'],Cortland County,NY,13.0,607,30112.0,283.0,106.58,0.34,12112.0,11217.0,108400.0,43985.0,-76.323463,-76.06199,42.732419,42.469917
2,043658949,SONS OF ITALY IN AMERICA,% STEPHANIE DENSMORE,176 ELM ST,CORTLAND,NY,13045-2322,3610,8,9,1000,194106,2,00,000000000,5,1,202312,0,0,2,0,12,0.0,0.0,0.0,Z99,1730 STELLA D ORO,1285,(Z) Unknown,This is a temporary code for organizations unt...,1,0,0,0,0,0,0,na,na,na,1941-06-01,(9) Subordinate (in Group),Association,code_not_found,Contributions not deductible.,13045,STANDARD,Cortland,"Cortland, NY",['Cortland'],Cortland County,NY,13.0,607,30112.0,283.0,106.58,0.34,12112.0,11217.0,108400.0,43985.0,-76.323463,-76.06199,42.732419,42.469917
3,115227037,CORTLANDVILLE FIRE DEPARTMENT INCORPORATED,NaN,999 NYS ROUTE 13,CORTLAND,NY,13045-3548,0000,3,3,1000,198602,1,15,407994000,1,1,202212,4,3,1,0,12,220657.0,50438.0,49043.0,M24Z,NaN,14649,"(M) Public Safety, Disaster Preparedness & Relief",Organizations that are responsible for the con...,1,0,0,0,407,994,0,Voluntary firemen's organization or auxiliary,code_not_found,na,1986-02-01,Independent,Corporation,Receives support from Gov or Public,Contributions deductible.,13045,STANDARD,Cortland,"Cortland, NY",['Cortland'],Cortland County,NY,13.0,607,30112.0,283.0,106.58,0.34,12112.0,11217.0,108400.0,43985.0,-76.323463,-76.06199,42.732419,42.469917
4,132951986,1890 HOUSE MUSEUM AND CENTER FOR THE ARTS,NaN,37 TOMPKINS STREET,CORTLAND,NY,13045-0000,0000,3,3,1000,197810,1,15,060000000,1,1,202212,4,3,1,0,12,438205.0,74274.0,74274.0,A82Z,NaN,18383,"(A) Arts, Culture & Humanities","Organizations that interpret, collect and pres...",1,0,0,0,60,0,0,"Museum, zoo, planetarium, etc.",na,na,1978-10-01,Independent,Corporation,Receives support from Gov or Public,Contributions deductible.,13045,STANDARD,Cortland,"Cortland, NY",['Cortland'],Cortland County,NY,13.0,607,30112.0,283.0,106.58,0.34,12112.0,11217.0,108400.0,43985.0,-76.323463,-76.06199,42.732419,42.469917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
# @title merge into np_local_df as the all-inclusive df for analysis
proc_dir = '/content/drive/My Drive/IRS_processed/'

np_local_df = np_cortland_p_df.merge(IRS_latest_df, on='EIN', how='left')

np_local_df = np_local_df.merge(cort_geo_df, on='p_org_id', how='left')

np_local_df = np_local_df.merge(web_srch_df, on='p_org_id', how='left')

# save final version for analysis/streamlit
np_local_df.to_csv(proc_dir + 'np_local_df.csv', index=False)

display(np_local_df)



,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,p_org_id,ntee_cat,ntee_define,class_cd_1,class_cd_2,class_cd_3,class_cd_4,activity_cd_1,activity_cd_2,activity_cd_3,act1_lu,act2_lu,act3_lu,RULE_DT,aff_lu,org_lu,found_lu,deduct_lu,zipcode,zipcode_type,major_city,post_office_city,...,GroupReturnForAffiliatesInd,Organization501c3Ind,TotalExpensesAmt,Pgm_Serv_Accomps,Total_Revenue,Mission,people,FormationYr,ActivityOrMissionDesc,TotalEmployeeCnt,MissionDesc,CYTotalRevenueAmt,Desc,status,coord_x,coord_y,cb_BASENAME,cb_BLKGRP,cb_BLOCK,cb_GEOID,cb_NAME,cb_TRACT,cb_COUNTY,cb_STATE,cnty_COUNTY,cnty_NAME,cnty_STATE,cong_BASENAME,cong_NAME,cntysub_NAME,cntysub_COUSUB,cntysub_GEOID,inc_PLACE,centracts_NAME,centracts_TRACT,csa_NAME,csa_GEOID,legup_NAME,leglow_NAME,num_matches,cntysub_PLACE,congress_rep_link,state_leg_lower_link,state_leg_upper_link,Unnamed: 0,url,found_name,snippet,full_result,ein
0,010939780,FREE & ACCEPTED MASONS OF NEW YORK,% JASON C TRABUCCO,15 EXCELSIOR ST,CORTLAND,NY,13045-2301,0265,10,9,1000,194012,2,00,279036000,5,1,202312,0,0,2,0,12,0.0,0.0,0.0,NaN,438 MARATHON LODGE,449,no_NTEE,no_NTEE,1,0,0,0,279,36,0,Other services or benefits to members or emplo...,Fraternity or sorority; Note: Key club (use 323),na,1940-12-01,(9) Subordinate (in Group),Association,code_not_found,Contributions not deductible.,13045,STANDARD,Cortland,"Cortland, NY",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-76.165818,42.602655,1012.0,1.0,1012.0,3.602397e+14,Block 1012,970500.0,23.0,36.0,23.0,Cortland County,36.0,19.0,Congressional District 19,Cortland city,18388.0,3.602318e+09,18388.0,Census Tract 9705,970500.0,"Ithaca-Cortland, NY CSA",296.0,State Senate District 52,Assembly District 125,1.0,NaN,"<a href=""https://clerk.house.gov/members/M0012...","<a href=""https://www.assembly.state.ny.us/mem/...","<a href=""https://www.nysenate.gov/district/52""...",0.0,https://nymasons.org/ny-lodges/,Lodge Locator - Grand Lodge of Free &amp; Acce...,Find a Lodge near you. With more than 400 loca...,"{""_type"": ""SearchResponse"", ""queryContext"": {""...",10939780.0
1,030469943,DISABLED AMERICAN VETERANS AUXILIARY INC,% CAROL WHITMARSH,14813 STATE ROUTE 90,CORTLAND,NY,13045-0000,2473,4,9,1000,197310,1,00,908921000,5,1,202306,0,0,2,0,6,0.0,0.0,0.0,NaN,153 CORTLAND MEMORIAL,949,no_NTEE,no_NTEE,1,0,0,0,908,921,0,Patriotic activities,Loans or credit reporting,na,1973-10-01,(9) Subordinate (in Group),Association,code_not_found,Contributions deductible.,13045,STANDARD,Cortland,"Cortland, NY",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-76.269550,42.640647,2036.0,2.0,2036.0,3.601104e+14,Block 2036,40900.0,11.0,36.0,11.0,Cayuga County,36.0,24.0,Congressional District 24,Summerhill town,72037.0,3.601172e+09,NaN,Census Tract 409,40900.0,"Syracuse-Auburn, NY CSA",532.0,State Senate District 48,Assembly District 131,1.0,na,"<a href=""https://clerk.house.gov/members/T0004...","<a href=""https://www.assembly.state.ny.us/mem/...","<a href=""https://www.nysenate.gov/district/48""...",1.0,https://www.cortlandnydav153.org/auxiliary/,Auxiliary | Disabled American Veterans – Cortl...,Contact an officer at the DAVA at 607-756-5100...,"{""_type"": ""SearchResponse"", ""queryContext"": {""...",30469943.0
2,043658949,SONS OF ITALY IN AMERICA,% STEPHANIE DENSMORE,176 ELM ST,CORTLAND,NY,13045-2322,3610,8,9,1000,194106,2,00,000000000,5,1,202312,0,0,2,0,12,0.0,0.0,0.0,Z99,1730 STELLA D ORO,1285,(Z) Unknown,This is a temporary code for organizations unt...,1,0,0,0,0,0,0,na,na,na,1941-06-01,(9) Subordinate (in Group),Association,code_not_found,Contributions not deductible.,13045,STANDARD,Cortland,"Cortland, NY",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-76.161390,42.602587,1008.0,1.0,1008.0,3.602397e+14,Block 1008,970500.0,23

In [ ]:
#np_local_df = np_cortland_p_df.merge(IRS_latest_df, on='EIN', how='left')

np_local_df.info(verbose=True)
#np_cortland_p_df.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 103 columns):
 #    Column                        Dtype  
---   ------                        -----  
 0    EIN                           object 
 1    NAME                          object 
 2    ICO                           object 
 3    STREET                        object 
 4    CITY                          object 
 5    STATE                         object 
 6    ZIP                           object 
 7    GROUP                         object 
 8    SUBSECTION                    int64  
 9    AFFILIATION                   object 
 10   CLASSIFICATION                object 
 11   RULING                        object 
 12   DEDUCTIBILITY                 int64  
 13   FOUNDATION                    object 
 14   ACTIVITY                      object 
 15   ORGANIZATION                  object 
 16   STATUS                        int64  
 17   TAX_PERIOD                    object 
 18   ASSET_CD

In [ ]:
# save np_local

# save final version for analysis/streamlit
np_local_df.to_csv(proc_dir + 'np_local_df.csv', index=False)




In [ ]:
# just load np_local_df

dtype = {"CLASSIFICATION": str,
         "EIN" : str,
         "ACTIVITY" : str,
         "AFFILIATION" : str,
         "ORGANIZATION" : str,
         "FOUNDATION" : str,
         "NTEE_CD" : str,
         "RULING" : str,
         "ZIP" : str,
         "TAX_PERIOD" : str,
         "GROUP" : str
         }

#TODO:check read to set dtypes properly
np_local_df = pd.read_csv('/content/drive/My Drive/IRS_processed/np_cortland_p_df.csv', dtype=dtype)

#dtype = {"EIN" : str,
#         "ZIPCd" : str
#         }




In [ ]:
#display (np_local_df)
np_local_df.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 114 columns):
 #    Column                        Dtype  
---   ------                        -----  
 0    EIN                           object 
 1    NAME                          object 
 2    ICO                           object 
 3    STREET                        object 
 4    CITY                          object 
 5    STATE                         object 
 6    ZIP                           object 
 7    GROUP                         object 
 8    SUBSECTION                    int64  
 9    AFFILIATION                   object 
 10   CLASSIFICATION                object 
 11   RULING                        object 
 12   DEDUCTIBILITY                 int64  
 13   FOUNDATION                    object 
 14   ACTIVITY                      object 
 15   ORGANIZATION                  object 
 16   STATUS                        int64  
 17   TAX_PERIOD                    object 
 18   ASSET_CD

# congressional district lookups - Moved, yes?

In [ ]:
# quick save as json to read back and use as lookup
import json

with open('/content/drive/My Drive/IRS_processed/congress.json', 'w') as cong:
    json.dump(district_dict, cong)

In [ ]:
# verify, read the json into dict

cong_dist = open('/content/drive/My Drive/IRS_processed/congress.json')

# returns JSON object as
# a dictionary
data = json.load(cong_dist)
pprint.pprint (data)


{'Congressional District 19': {'bioguideId': 'M001221',
                               'house_url': 'https://clerk.house.gov/members/M001221',
                               'name': 'Molinaro, Marcus J.',
                               'partyName': 'Republican'},
 'Congressional District 24': {'bioguideId': 'T000478',
                               'house_url': 'https://clerk.house.gov/members/T000478',
                               'name': 'Tenney, Claudia',
                               'partyName': 'Republican'}}


# Prepping for Mapping

- Some orgs are co-located or very close which makes map markers unusable
- Some orgs submit to IRS with PO box which can't be geocoded. (TODO:Another round of websearch to get orgs with no address)
- can use latlng or street address to find close


In [ ]:
# load the dataset, if needed

np_local_df = pd.read_csv('/content/drive/My Drive/IRS_processed/np_local_df.csv')

display(np_local_df)


,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,p_org_id,ntee_cat,ntee_define,CLASS1,CLASS2,CLASS3,CLASS4,ACT1,ACT2,ACT3,act1_lu,act2_lu,act3_lu,RULE_DT,aff_lu,org_lu,found_lu,filename,IRS_Form,TaxPeriodEndDt,TaxPeriodBeginDt,TaxYr,...,Total_Revenue,Mission,people,FormationYr,ActivityOrMissionDesc,TotalEmployeeCnt,MissionDesc,CYTotalRevenueAmt,Desc,status,coord_x,coord_y,cb_BASENAME,cb_BLKGRP,cb_BLOCK,cb_GEOID,cb_NAME,cb_TRACT,cb_COUNTY,cb_STATE,cnty_COUNTY,cnty_NAME,cnty_STATE,cong_BASENAME,cong_NAME,cntysub_NAME,cntysub_COUSUB,cntysub_GEOID,inc_PLACE,centracts_NAME,centracts_TRACT,csa_NAME,csa_GEOID,legup_NAME,leglow_NAME,num_matches,cntysub_PLACE,congress_rep_link,state_leg_lower_link,state_leg_upper_link,Unnamed: 0,url,found_name,snippet,full_result,ein,cluster_lng,cluster_lat,cluster_ind,cluster_ngroup
0,132951986,1890 HOUSE MUSEUM AND CENTER FOR THE ARTS,NaN,37 TOMPKINS STREET,CORTLAND,NY,13045-0000,0,3,3,1000,197810,1,15,60000000,1,1,202212.0,4,3,1,0,12,438205.0,74274.0,74274.0,A82Z,NaN,18383,"(A) Arts, Culture & Humanities","Organizations that interpret, collect and pres...",1,0,0,0,60,0,0,"Museum, zoo, planetarium, etc.",na,na,1978-10-01,Independent - This code is used if the organiz...,Corporation,Organization which receives a substantial part...,202313179349307826_public.xml,IRS990,2022-12-31,2022-01-01,2022.0,...,74274,(f990) PRESERVE ITEMS OF THE 1890S AND THE VIC...,"[{'PersonNm': 'TERRY MINGLE', 'TitleTxt': 'BD ...",1984,PRESERVE ITEMS OF THE 1890S AND THE VICTORIAN ...,0,PRESERVE ITEMS OF THE 1890S AND THE VICTORIAN ...,74274,COLLECTION ACCESSION - TO ACQUIRE VICTORIANFUR...,success,-76.182607,42.596112,3001.0,3.0,3001.0,3.602397e+14,Block 3001,970900.0,23.0,36.0,23.0,Cortland County,36.0,19.0,Congressional District 19,Cortland city,18388.0,3.602318e+09,18388.0,Census Tract 9709,970900.0,"Ithaca-Cortland, NY CSA",296.0,State Senate District 52,Assembly District 125,1.0,NaN,"<a href=""https://clerk.house.gov/members/M0012...","<a href=""https://www.assembly.state.ny.us/mem/...","<a href=""https://www.nysenate.gov/district/52""...",1.0,https://the1890house.org/,"The 1890 House, Cortland, NY - The 1890 House ...",The impressive collection found throughout the...,"{""_type"": ""SearchResponse"", ""queryContext"": {""...",132951986.0,-76.183,42.596,0.0,36.0
1,161557965,ACCESS TO INDEPENDENCE OF CORTLAND COUNTY INC,NaN,26 NORTH MAIN STREET,CORTLAND,NY,13045-2198,0,3,3,2000,199910,1,15,0,1,1,202309.0,5,5,1,0,9,534363.0,842554.0,842554.0,B80,NaN,43730,(B) Education,"Organizations that provide services, facilitie...",2,0,0,0,0,0,0,na,na,na,1999-10-01,Independent - This code is used if the organiz...,Corporation,Organization which receives a substantial part...,202441169349302154_public.xml,IRS990,2023-09-30,2022-10-01,2022.0,...,842554,(f990) THE MISSION OF ACCESS TO INDEPENDENCE I...,"[{'PersonNm': 'Alexandra Mikowski', 'TitleTxt'...",1998,THE MISSION OF ACCESS TO INDEPENDENCE IS TO EM...,16,THE MISSION OF ACCESS TO INDEPENDENCE IS TO EM...,842554,THE MISSION OF ACCESS TO INDEPENDENCE IS TO EM...,success,-76.180692,42.604318,3002.0,3.0,3002.0,3.602397e+14,Block 3002,970500.0,23.0,36.0,23.0,Cortland County,36.0,19.0,Congressional District 19,Cortland city,18388.0,3.602318e+09,18388.0,Census Tract 9705,970500.0,"Ithaca-Cortland, NY CSA",296.0,State Senate District 52,Assembly District 125,1.0,NaN,"<a href=""https://clerk.house.gov/members/M0012...","<a href=""https://www.assembly.state.ny.us/mem/...","<a href=""https://www.nysenate.gov/district/52""...",NaN,NaN,NaN,NaN,NaN,NaN,-76.181,42.604,1.0,68.0
2,204703107,AFRICA READS INC,% TINA CAVALIER,15 ELLWOOD AVE,CORTLAND,NY,13045-1807,0,3,3,1000,200805,1,15,0,1,1,202403.0,0,0,2,0,3,0.0,0.0,0.0,Q30,NaN,49916,"(Q) International, Foreign Affairs & National ...",Organi

In [ ]:
# sorta alphabetically and reset index to get sequential number
# will be useful for mapping

np_local_df.sort_values(by=['NAME'], inplace=True)
np_local_df.reset_index(drop=True, inplace=True)

# start index at 1 for humans, when matching org to map number
np_local_df.index += 1

display(np_local_df)

,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,p_org_id,ntee_cat,ntee_define,class_cd_1,class_cd_2,class_cd_3,class_cd_4,activity_cd_1,activity_cd_2,activity_cd_3,act1_lu,act2_lu,act3_lu,RULE_DT,aff_lu,org_lu,found_lu,deduct_lu,zipcode,zipcode_type,major_city,post_office_city,...,GroupReturnForAffiliatesInd,Organization501c3Ind,TotalExpensesAmt,Pgm_Serv_Accomps,Total_Revenue,Mission,people,FormationYr,ActivityOrMissionDesc,TotalEmployeeCnt,MissionDesc,CYTotalRevenueAmt,Desc,status,coord_x,coord_y,cb_BASENAME,cb_BLKGRP,cb_BLOCK,cb_GEOID,cb_NAME,cb_TRACT,cb_COUNTY,cb_STATE,cnty_COUNTY,cnty_NAME,cnty_STATE,cong_BASENAME,cong_NAME,cntysub_NAME,cntysub_COUSUB,cntysub_GEOID,inc_PLACE,centracts_NAME,centracts_TRACT,csa_NAME,csa_GEOID,legup_NAME,leglow_NAME,num_matches,cntysub_PLACE,congress_rep_link,state_leg_lower_link,state_leg_upper_link,Unnamed: 0,url,found_name,snippet,full_result,ein
1,132951986,1890 HOUSE MUSEUM AND CENTER FOR THE ARTS,NaN,37 TOMPKINS STREET,CORTLAND,NY,13045-0000,0000,3,3,1000,197810,1,15,060000000,1,1,202212,4,3,1,0,12,438205.0,74274.0,74274.0,A82Z,NaN,18383,"(A) Arts, Culture & Humanities","Organizations that interpret, collect and pres...",1,0,0,0,60,0,0,"Museum, zoo, planetarium, etc.",na,na,1978-10-01,Independent,Corporation,Receives support from Gov or Public,Contributions deductible.,13045,STANDARD,Cortland,"Cortland, NY",...,false,X,tag_not_found,"{'Desc': 'ADMISSIONS', 'BusinessCd': '900099',...",74274,(f990) PRESERVE ITEMS OF THE 1890S AND THE VIC...,"[{'PersonNm': 'TERRY MINGLE', 'TitleTxt': 'BD ...",1984,PRESERVE ITEMS OF THE 1890S AND THE VICTORIAN ...,0,PRESERVE ITEMS OF THE 1890S AND THE VICTORIAN ...,74274,COLLECTION ACCESSION - TO ACQUIRE VICTORIANFUR...,success,-76.182607,42.596112,3001.0,3.0,3001.0,3.602397e+14,Block 3001,970900.0,23.0,36.0,23.0,Cortland County,36.0,19.0,Congressional District 19,Cortland city,18388.0,3.602318e+09,18388.0,Census Tract 9709,970900.0,"Ithaca-Cortland, NY CSA",296.0,State Senate District 52,Assembly District 125,1.0,NaN,"<a href=""https://clerk.house.gov/members/M0012...","<a href=""https://www.assembly.state.ny.us/mem/...","<a href=""https://www.nysenate.gov/district/52""...",1.0,https://the1890house.org/,"The 1890 House, Cortland, NY - The 1890 House ...",The impressive collection found throughout the...,"{""_type"": ""SearchResponse"", ""queryContext"": {""...",132951986.0
2,161557965,ACCESS TO INDEPENDENCE OF CORTLAND COUNTY INC,NaN,26 NORTH MAIN STREET,CORTLAND,NY,13045-2198,0000,3,3,2000,199910,1,15,000000000,1,1,202309,5,5,1,0,9,534363.0,842554.0,842554.0,B80,NaN,43730,(B) Education,"Organizations that provide services, facilitie...",2,0,0,0,0,0,0,na,na,na,1999-10-01,Independent,Corporation,Receives support from Gov or Public,Contributions deductible.,13045,STANDARD,Cortland,"Cortland, NY",...,false,X,tag_not_found,tag_not_found,842554,(f990) THE MISSION OF ACCESS TO INDEPENDENCE I...,"[{'PersonNm': 'Alexandra Mikowski', 'TitleTxt'...",1998,THE MISSION OF ACCESS TO INDEPENDENCE IS TO EM...,16,THE MISSION OF ACCESS TO INDEPENDENCE IS TO EM...,842554,THE MISSION OF ACCESS TO INDEPENDENCE IS TO EM...,success,-76.180692,42.604318,3002.0,3.0,3002.0,3.602397e+14,Block 3002,970500.0,23.0,36.0,23.0,Cortland County,36.0,19.0,Congressional District 19,Cortland city,18388.0,3.602318e+09,18388.0,Census Tract 9705,970500.0,"Ithaca-Cortland, NY CSA",296.0,State Senate District 52,Assembly District 125,1.0,NaN,"<a href=""https://clerk.house.gov/members/M0012...","<a href=""https://www.assembly.state.ny.us/mem/...","<a href=""https://www.nysenate.gov/district/52""...",NaN,NaN,NaN,NaN,NaN,NaN
3,204703107,AFRICA READS INC,% TINA CAVALIER,15 ELLWOOD AVE,CORTLAND,NY,13045-1807,0000,3,3,1000,200805,1,15,000000000,1,1,202403,0,0,2,0,3,0.0,0.0,0.0,Q30,NaN,49916,"(Q)

## Colocated or close marker tests

For scenarios where nonprofits are either colocated with other Nonprofits or very close, mapping gets clutterered.  These cells identify those scenarios and add a data element indicate the "cluster" and a counter for each Nonprofit in the cluster.

In [ ]:
# Investigate Issue:  orgs that are very close have overlapping pins
# can use cluster marker or
# https://stackoverflow.com/questions/72874876/folium-markers-popups-from-data-with-duplicated-lat-long



,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,p_org_id,ntee_cat,ntee_define,CLASS1,CLASS2,CLASS3,CLASS4,ACT1,ACT2,ACT3,act1_lu,act2_lu,act3_lu,RULE_DT,aff_lu,org_lu,found_lu,filename,IRS_Form,TaxPeriodEndDt,TaxPeriodBeginDt,TaxYr,...,GroupReturnForAffiliatesInd,Organization501c3Ind,TotalExpensesAmt,Pgm_Serv_Accomps,Total_Revenue,Mission,people,FormationYr,ActivityOrMissionDesc,TotalEmployeeCnt,MissionDesc,CYTotalRevenueAmt,Desc,status,coord_x,coord_y,cb_BASENAME,cb_BLKGRP,cb_BLOCK,cb_GEOID,cb_NAME,cb_TRACT,cb_COUNTY,cb_STATE,cnty_COUNTY,cnty_NAME,cnty_STATE,cong_BASENAME,cong_NAME,cntysub_NAME,cntysub_COUSUB,cntysub_GEOID,inc_PLACE,centracts_NAME,centracts_TRACT,csa_NAME,csa_GEOID,legup_NAME,leglow_NAME,num_matches,cntysub_PLACE,congress_rep_link,state_leg_lower_link,state_leg_upper_link,Unnamed: 0,url,found_name,snippet,full_result,ein
0,10939780,FREE & ACCEPTED MASONS OF NEW YORK,% JASON C TRABUCCO,15 EXCELSIOR ST,CORTLAND,NY,13045-2301,265,10,9,1000,194012,2,0,279036000,5,1,202312.0,0,0,2,0,12,0.0,0.0,0.0,NaN,438 MARATHON LODGE,449,no_NTEE,no_ntee,1,0,0,0,279,36,0,Other services or benefits to members or emplo...,Fraternity or sorority; Note: Key club (use 323),na,1940-12-01,Subordinate - This code is used if the organiz...,Association,code_not_found,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-76.165818,42.602655,1012.0,1.0,1012.0,3.602397e+14,Block 1012,970500.0,23.0,36.0,23.0,Cortland County,36.0,19.0,Congressional District 19,Cortland city,18388.0,3.602318e+09,18388.0,Census Tract 9705,970500.0,"Ithaca-Cortland, NY CSA",296.0,State Senate District 52,Assembly District 125,1.0,NaN,"<a href=""https://clerk.house.gov/members/M0012...","<a href=""https://www.assembly.state.ny.us/mem/...","<a href=""https://www.nysenate.gov/district/52""...",0.0,https://nymasons.org/ny-lodges/,Lodge Locator - Grand Lodge of Free &amp; Acce...,Find a Lodge near you. With more than 400 loca...,"{""_type"": ""SearchResponse"", ""queryContext"": {""...",10939780.0
1,30469943,DISABLED AMERICAN VETERANS AUXILIARY INC,% CAROL WHITMARSH,14813 STATE ROUTE 90,CORTLAND,NY,13045-0000,2473,4,9,1000,197310,1,0,908921000,5,1,202306.0,0,0,2,0,6,0.0,0.0,0.0,NaN,153 CORTLAND MEMORIAL,949,no_NTEE,no_ntee,1,0,0,0,908,921,0,Patriotic activities,Loans or credit reporting,na,1973-10-01,Subordinate - This code is used if the organiz...,Association,code_not_found,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-76.269550,42.640647,2036.0,2.0,2036.0,3.601104e+14,Block 2036,40900.0,11.0,36.0,11.0,Cayuga County,36.0,24.0,Congressional District 24,Summerhill town,72037.0,3.601172e+09,NaN,Census Tract 409,40900.0,"Syracuse-Auburn, NY CSA",532.0,State Senate District 48,Assembly District 131,1.0,na,"<a href=""https://clerk.house.gov/members/T0004...","<a href=""https://www.assembly.state.ny.us/mem/...","<a href=""https://www.nysenate.gov/district/48""...",1.0,https://www.cortlandnydav153.org/auxiliary/,Auxiliary | Disabled American Veterans – Cortl...,Contact an officer at the DAVA at 607-756-5100...,"{""_type"": ""SearchResponse"", ""queryContext"": {""...",30469943.0
2,43658949,SONS OF ITALY IN AMERICA,% STEPHANIE DENSMORE,176 ELM ST,CORTLAND,NY,13045-2322,3610,8,9,1000,194106,2,0,0,5,1,202312.0,0,0,2,0,12,0.0,0.0,0.0,Z99,1730 STELLA D ORO,1285,(Z) Unknown,This is a temporary code for organizations unt...,1,0,0,0,0,0,0,na,na,na,1941-06-01,Subordinate - This code is used if the organiz...,Association,code_not_found,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-76.161390,42.602587,1008.0,1.0,1008.0,3.602397e+14,Block 1008,970500.0,23.0,36.0,23.0,Cortland County,36.0,19.0,Congressional District 19,Cortland city,18388.0,3.602318e+09,18388.0,Census Tract 9705,970

In [ ]:
# First:  Create a cluster lat lng
# create a new column to indicate clustering
# clustering would be roughly the same lat and lng

#TODO: rename coords as lat and lng
#TODO: consider convert to geopandas and point geometry?

# create a rounded coord_x
np_local_df['cluster_lng'] = np_local_df['coord_x'].round(3)
np_local_df['cluster_lat'] = np_local_df['coord_y'].round(3)

np_local_df[['NAME', 'STREET', 'cluster_lng', 'cluster_lat', 'cluster_ind']].sort_values(by=['cluster_lat', 'cluster_ind', 'cluster_ind'])


KeyError: "['cluster_ind'] not in index"

In [ ]:
display(np_local_df[['NAME', 'STREET', 'coord_x', 'cluster_lng', 'coord_y', 'cluster_lat']])


,NAME,STREET,coord_x,cluster_lng,coord_y,cluster_lat
0,1890 HOUSE MUSEUM AND CENTER FOR THE ARTS,37 TOMPKINS STREET,-76.182607,-76.183,42.596112,42.596
1,ACCESS TO INDEPENDENCE OF CORTLAND COUNTY INC,26 NORTH MAIN STREET,-76.180692,-76.181,42.604318,42.604
2,AFRICA READS INC,15 ELLWOOD AVE,-76.196607,-76.197,42.600015,42.600
3,ALPHA PHI OMEGA,406 COREY UNION RM 412,NaN,NaN,NaN,NaN
4,AMERICAN FEDERATION OF TEACHERS,781 STATE ROUTE 392,-76.223742,-76.224,42.494106,42.494
...,...,...,...,...,...,...
157,YAMAN FOUNDATION,165 MAIN ST STE 1,-76.180212,-76.180,42.597786,42.598
158,YMCA OF CORTLAND PROPERTIES INC,22 TOMPKINS STREET,-76.181616,-76.182,42.596879,42.597
159,YOUNG MENS CHRISTIAN ASSOCIATION CORTLAND,22 TOMPKINS ST,-76.181616,-76.182,42.596879,42.597
160,YOUNG WOMENS CHRISTIAN ASSOCIATION,14 CLAYTON AVE,-76.181466,-76.181,42.598056,42.598


In [ ]:
# find dupes of cluster lat / lng
# find orgs with same street address
filt = np_local_df.duplicated(subset=['cluster_lat', 'cluster_lng'], keep=False)
# np_local_df[filt].sort_values(by=['cluster_lat', 'cluster_lng'])

# np_local_df[['NAME', 'STREET', 'coord_x', 'cluster_lng', 'coord_y', 'cluster_lat']]

np_local_df[filt][['NAME', 'STREET', 'coord_x', 'cluster_lng', 'coord_y', 'cluster_lat']].sort_values(by=['cluster_lat', 'cluster_lng'])



,NAME,STREET,coord_x,cluster_lng,coord_y,cluster_lat
122,NEW APOSTOLIC CHURCH OF NORWICH NY,3710 LYNCOURT DR,-76.234217,-76.234,42.585467,42.585
123,NEW APOSTOLIC CHURCH OF SYRACUSE NY,3710 LYNCOURT DR,-76.234217,-76.234,42.585467,42.585
5,AMERICAN LEGION,212 TOMPKINS ST,-76.196960,-76.197,42.587428,42.587
6,AMERICAN LEGION AUXILIARY,212 TOMPKINS ST,-76.196960,-76.197,42.587428,42.587
61,CORTLAND JR SR HIGH SCHOOL MUSIC BOOSTERS CLUB,1 VALLEY VIEW DR,-76.225059,-76.225,42.591270,42.591
68,CORTLAND PUBLIC EDUCATION FOUNDATION INCORPORATED,1 VALLEY VIEW DR,-76.225059,-76.225,42.591270,42.591
8,ASC FAMILY FUND INC,15 NEUBIG RD,-76.193629,-76.194,42.595477,42.595
10,AUXILIARY SERVICES CORPORATION OF SUNY CORTLAND,15 NEUBIG RD,-76.193629,-76.194,42.595477,42.595
158,YMCA OF CORTLAND PROPERTIES INC,22 TOMPKINS STREET,-76.181616,-76.182,42.596879,42.597
159,YOUNG MENS CHRISTIAN ASSOCIATION CORTLAND,22 TOMPKINS ST,-76.181616,-76.182,42.596879,42.597


In [ ]:
display(np_local_df[['NAME', 'STREET', 'coord_x', 'cluster_lng',
                     'coord_y', 'cluster_lat', 'cluster_ind']])


,NAME,STREET,coord_x,cluster_lng,coord_y,cluster_lat,cluster_ind
0,1890 HOUSE MUSEUM AND CENTER FOR THE ARTS,37 TOMPKINS STREET,-76.182607,-76.183,42.596112,42.596,0.0
1,ACCESS TO INDEPENDENCE OF CORTLAND COUNTY INC,26 NORTH MAIN STREET,-76.180692,-76.181,42.604318,42.604,0.0
2,AFRICA READS INC,15 ELLWOOD AVE,-76.196607,-76.197,42.600015,42.600,0.0
3,ALPHA PHI OMEGA,406 COREY UNION RM 412,NaN,NaN,NaN,NaN,NaN
4,AMERICAN FEDERATION OF TEACHERS,781 STATE ROUTE 392,-76.223742,-76.224,42.494106,42.494,0.0
...,...,...,...,...,...,...,...
157,YAMAN FOUNDATION,165 MAIN ST STE 1,-76.180212,-76.180,42.597786,42.598,1.0
158,YMCA OF CORTLAND PROPERTIES INC,22 TOMPKINS STREET,-76.181616,-76.182,42.596879,42.597,0.0
159,YOUNG MENS CHRISTIAN ASSOCIATION CORTLAND,22 TOMPKINS ST,-76.181616,-76.182,42.596879,42.597,1.0
160,YOUNG WOMENS CHRISTIAN ASSOCIATION,14 CLAYTON AVE,-76.181466,-76.181,42.598056,42.598,1.0


## Create Cluster Group

In [ ]:
# create some indicator of clustering
# where close (rounded) lat/lat have multiple markers, count

# look at data
# np_local_df.groupby(['coord_x', 'coord_y']).count()
# np_local_df.groupby(['cluster_lat', 'cluster_lng']).count()
# np_local_df.groupby(['cluster_lat', 'cluster_lng']).size()

# gives cumcount is 0 for first dupe *and* for non-duped
# could use filter to only get duplicated rounded lat/lng
# or could follow- up by updated

# df['cluster_ind'] = df.groupby('orgid').cumcount()
np_local_df['cluster_ind'] = np_local_df.groupby(['cluster_lat', 'cluster_lng']).cumcount() + 1

np_local_df['cluster_ngroup'] = np_local_df.groupby(['cluster_lat', 'cluster_lng']).ngroup()


In [ ]:
display(np_local_df[['NAME', 'STREET', 'coord_x', 'cluster_lng', 'coord_y',
                     'cluster_lat', 'cluster_ind', 'cluster_ngroup']])




,NAME,STREET,coord_x,cluster_lng,coord_y,cluster_lat,cluster_ind,cluster_ngroup
1,1890 HOUSE MUSEUM AND CENTER FOR THE ARTS,37 TOMPKINS STREET,-76.182607,-76.183,42.596112,42.596,1.0,36.0
2,ACCESS TO INDEPENDENCE OF CORTLAND COUNTY INC,26 NORTH MAIN STREET,-76.180692,-76.181,42.604318,42.604,1.0,68.0
3,AFRICA READS INC,15 ELLWOOD AVE,-76.196607,-76.197,42.600015,42.600,1.0,50.0
4,ALPHA PHI OMEGA,406 COREY UNION RM 412,NaN,NaN,NaN,NaN,NaN,NaN
5,AMERICAN FEDERATION OF TEACHERS,781 STATE ROUTE 392,-76.223742,-76.224,42.494106,42.494,1.0,1.0
...,...,...,...,...,...,...,...,...
158,YAMAN FOUNDATION,165 MAIN ST STE 1,-76.180212,-76.180,42.597786,42.598,2.0,41.0
159,YMCA OF CORTLAND PROPERTIES INC,22 TOMPKINS STREET,-76.181616,-76.182,42.596879,42.597,1.0,38.0
160,YOUNG MENS CHRISTIAN ASSOCIATION CORTLAND,22 TOMPKINS ST,-76.181616,-76.182,42.596879,42.597,2.0,38.0
161,YOUNG WOMENS CHRISTIAN ASSOCIATION,14 CLAYTON AVE,-76.181466,-76.181,42.598056,42.598,2.0,40.0


In [ ]:

# get all rows that do not have duplicated cluster lat/lng
filt = ~(np_local_df.duplicated(subset=['cluster_lat', 'cluster_lng'], keep=False))
# np_local_df[filt].sort_values(by=['cluster_lat', 'cluster_lng'])

# review
# np_local_df[['NAME', 'STREET', 'coord_x', 'cluster_lng', 'coord_y', 'cluster_lat']]

# review with filter
#np_local_df[filt][['NAME', 'STREET', 'cluster_lng',
#                   'cluster_lat', 'cluster_ind', 'cluster_ngroup']].sort_values(by=['cluster_lat', 'cluster_lng'])

np_local_df.loc[filt, 'cluster_ind'] = 0



In [ ]:
#display(np_local_df.sort_values(by=['cluster_lat', 'cluster_lng']))

display(np_local_df[['NAME', 'STREET',
                     'cluster_lat', 'cluster_lng',
                     'cluster_ind', 'cluster_ngroup']].head(50))
# .sort_values(by=['cluster_lat', 'cluster_lng'])



,NAME,STREET,cluster_lat,cluster_lng,cluster_ind,cluster_ngroup
0,1890 HOUSE MUSEUM AND CENTER FOR THE ARTS,37 TOMPKINS STREET,42.596,-76.183,0.0,36.0
1,ACCESS TO INDEPENDENCE OF CORTLAND COUNTY INC,26 NORTH MAIN STREET,42.604,-76.181,1.0,68.0
2,AFRICA READS INC,15 ELLWOOD AVE,42.600,-76.197,0.0,50.0
3,ALPHA PHI OMEGA,406 COREY UNION RM 412,NaN,NaN,NaN,NaN
4,AMERICAN FEDERATION OF TEACHERS,781 STATE ROUTE 392,42.494,-76.224,0.0,1.0
5,AMERICAN LEGION,212 TOMPKINS ST,42.587,-76.197,1.0,24.0
6,AMERICAN LEGION AUXILIARY,212 TOMPKINS ST,42.587,-76.197,2.0,24.0
7,AMERICAN VOLKSSPORT ASSOCIATION INC,1041 CHURCH ST,42.506,-76.209,0.0,2.0
8,ASC FAMILY FUND INC,15 NEUBIG RD,42.595,-76.194,1.0,33.0
9,AUTOMOTIVE LIFT INSTITUTE INC,3699 LUKER RD,42.586,-76.219,0.0,21.0


## Save np_local with final tweaks

In [ ]:
# Save np_local with final tweaks

np_local_df.to_csv(proc_dir + 'np_local_df.csv', index=False)

display(np_local_df)


,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,p_org_id,ntee_cat,ntee_define,class_cd_1,class_cd_2,class_cd_3,class_cd_4,activity_cd_1,activity_cd_2,activity_cd_3,act1_lu,act2_lu,act3_lu,RULE_DT,aff_lu,org_lu,found_lu,deduct_lu,zipcode,zipcode_type,major_city,post_office_city,...,Total_Revenue,Mission,people,FormationYr,ActivityOrMissionDesc,TotalEmployeeCnt,MissionDesc,CYTotalRevenueAmt,Desc,status,coord_x,coord_y,cb_BASENAME,cb_BLKGRP,cb_BLOCK,cb_GEOID,cb_NAME,cb_TRACT,cb_COUNTY,cb_STATE,cnty_COUNTY,cnty_NAME,cnty_STATE,cong_BASENAME,cong_NAME,cntysub_NAME,cntysub_COUSUB,cntysub_GEOID,inc_PLACE,centracts_NAME,centracts_TRACT,csa_NAME,csa_GEOID,legup_NAME,leglow_NAME,num_matches,cntysub_PLACE,congress_rep_link,state_leg_lower_link,state_leg_upper_link,Unnamed: 0,url,found_name,snippet,full_result,ein,cluster_lng,cluster_lat,cluster_ind,cluster_ngroup
1,132951986,1890 HOUSE MUSEUM AND CENTER FOR THE ARTS,NaN,37 TOMPKINS STREET,CORTLAND,NY,13045-0000,0000,3,3,1000,197810,1,15,060000000,1,1,202212,4,3,1,0,12,438205.0,74274.0,74274.0,A82Z,NaN,18383,"(A) Arts, Culture & Humanities","Organizations that interpret, collect and pres...",1,0,0,0,60,0,0,"Museum, zoo, planetarium, etc.",na,na,1978-10-01,Independent,Corporation,Receives support from Gov or Public,Contributions deductible.,13045,STANDARD,Cortland,"Cortland, NY",...,74274,(f990) PRESERVE ITEMS OF THE 1890S AND THE VIC...,"[{'PersonNm': 'TERRY MINGLE', 'TitleTxt': 'BD ...",1984,PRESERVE ITEMS OF THE 1890S AND THE VICTORIAN ...,0,PRESERVE ITEMS OF THE 1890S AND THE VICTORIAN ...,74274,COLLECTION ACCESSION - TO ACQUIRE VICTORIANFUR...,success,-76.182607,42.596112,3001.0,3.0,3001.0,3.602397e+14,Block 3001,970900.0,23.0,36.0,23.0,Cortland County,36.0,19.0,Congressional District 19,Cortland city,18388.0,3.602318e+09,18388.0,Census Tract 9709,970900.0,"Ithaca-Cortland, NY CSA",296.0,State Senate District 52,Assembly District 125,1.0,NaN,"<a href=""https://clerk.house.gov/members/M0012...","<a href=""https://www.assembly.state.ny.us/mem/...","<a href=""https://www.nysenate.gov/district/52""...",1.0,https://the1890house.org/,"The 1890 House, Cortland, NY - The 1890 House ...",The impressive collection found throughout the...,"{""_type"": ""SearchResponse"", ""queryContext"": {""...",132951986.0,-76.183,42.596,0.0,36.0
2,161557965,ACCESS TO INDEPENDENCE OF CORTLAND COUNTY INC,NaN,26 NORTH MAIN STREET,CORTLAND,NY,13045-2198,0000,3,3,2000,199910,1,15,000000000,1,1,202309,5,5,1,0,9,534363.0,842554.0,842554.0,B80,NaN,43730,(B) Education,"Organizations that provide services, facilitie...",2,0,0,0,0,0,0,na,na,na,1999-10-01,Independent,Corporation,Receives support from Gov or Public,Contributions deductible.,13045,STANDARD,Cortland,"Cortland, NY",...,842554,(f990) THE MISSION OF ACCESS TO INDEPENDENCE I...,"[{'PersonNm': 'Alexandra Mikowski', 'TitleTxt'...",1998,THE MISSION OF ACCESS TO INDEPENDENCE IS TO EM...,16,THE MISSION OF ACCESS TO INDEPENDENCE IS TO EM...,842554,THE MISSION OF ACCESS TO INDEPENDENCE IS TO EM...,success,-76.180692,42.604318,3002.0,3.0,3002.0,3.602397e+14,Block 3002,970500.0,23.0,36.0,23.0,Cortland County,36.0,19.0,Congressional District 19,Cortland city,18388.0,3.602318e+09,18388.0,Census Tract 9705,970500.0,"Ithaca-Cortland, NY CSA",296.0,State Senate District 52,Assembly District 125,1.0,NaN,"<a href=""https://clerk.house.gov/members/M0012...","<a href=""https://www.assembly.state.ny.us/mem/...","<a href=""https://www.nysenate.gov/district/52""...",NaN,NaN,NaN,NaN,NaN,NaN,-76.181,42.604,1.0,68.0
3,204703107,AFRICA READS INC,% TINA CAVALIER,15 ELLWOOD AVE,CORTLAND,NY,13045-1807,0000,3,3,1000,200805,1,15,000000000,1,1,202403,0,0,2,0,3,0.0,0.0,0.0,Q30,NaN,49916,"(Q) International, Foreign Affairs & National ...",Organizations whose primary purpose is to prov

# Quick Checks, Scratchpad

In [ ]:
# validating

dtype = {"CLASSIFICATION": str,
         "EIN" : str,
         "ACTIVITY" : str,
         "AFFILIATION" : str,
         "ORGANIZATION" : str,
         "FOUNDATION" : str,
         "NTEE_CD" : str,
         "RULING" : str,
         "ZIP" : str,
         "TAX_PERIOD" : str,
         "GROUP" : str
         }

np_cortland_p_df = pd.read_csv('/content/drive/My Drive/IRS_processed/np_cortland_p_df.csv', dtype=dtype)


# Load Presentation lookups



In [ ]:
# loading dict to lookup cryptic columns with display info
#TODO: combine with IRS Lookups?

import csv
import pprint
present_lu = {}

with open('/content/drive/My Drive/IRS_processed/presentation_lookups.csv', mode='r') as infile:
    reader = csv.reader(infile)
    for row in reader:
      present_lu[row[1]] = [row[3], row[4], row[5], row[6], row[7], row[8]]

# key_name ['source', 'display_name', 'display_section', 'format', 'format_cd', 'help']
for key, value in present_lu.items():
  print (key, value)

#pprint.pprint(present_lu)



key_name ['source', 'display_name', 'display_section', 'format', 'format_cd', 'help']
EIN ['bmf', 'EIN', 'bmf', 'str', '', 'Tax ID for organizations (get formal def)']
NAME ['bmf', 'Org Name', 'bmf', 'str', '', 'Organization Name on IRS Form 990x']
ICO ['bmf', 'ICO', 'bmf', 'str', '', 'In Care Of']
STREET ['bmf', 'STREET', 'bmf', 'str', '', 'Postal Address']
CITY ['bmf', 'CITY', 'bmf', 'str', '', 'Postal Address']
STATE ['bmf', 'STATE', 'bmf', 'str', '', 'Postal Address']
ZIP ['bmf', 'ZIP', 'bmf', 'str', '', 'Postal Address']
GROUP ['bmf', 'Parent Org Code', 'bmf', 'str', '', 'This is an internal number assigned to central/parent organizations holding group determination letters.']
SUBSECTION ['bmf', 'SUBSECTION', 'bmf', 'str', '', 'Subsection Codes are the codes shown under section 501(c) of the Internal Revenue Code of 1986 which define the category under which an organization may be exempt. A table of subsection and classification codes can be found on page 6. From one to three subs

In [ ]:
# load presentation info into dict
#

import csv
import pprint
present_lu = {}

with open('/content/drive/My Drive/IRS_processed/presentation_lookups.csv', mode='r') as infile:
    reader = csv.reader(infile)
    row_cnt = 1
    for row in reader:
      if row_cnt == 1:
        keys = row
        row_cnt += 1
      else:
        present_lu[row[1]] = dict(zip(keys, row))
        row_cnt += 1

#      present_lu[row[1]] = [row[3], row[4], row[5], row[6], row[7], row[8]]

# key_name ['source', 'display_name', 'display_section', 'format', 'format_cd', 'help']
for key, value in present_lu.items():
  print (key, value)

for fld_dict in present_lu:
  del present_lu[fld_dict]['key_name']
  del present_lu[fld_dict]['sample']

for key, value in present_lu.items():
  print (key, value)


#pprint.pprint(present_lu


EIN {'original order': '1', 'key_name': 'EIN', 'sample': '204703107', 'source': 'bmf', 'display_name': 'EIN', 'display_section': 'bmf', 'format': 'str', 'format_cd': '', 'help': 'Tax ID for organizations (get formal def)', 'info_link': '', '': ''}
NAME {'original order': '2', 'key_name': 'NAME', 'sample': 'AFRICA READS INC', 'source': 'bmf', 'display_name': 'Org Name', 'display_section': 'bmf', 'format': 'title', 'format_cd': '', 'help': 'Organization Name on IRS Form 990x', 'info_link': '', '': ''}
ICO {'original order': '3', 'key_name': 'ICO', 'sample': '% TINA CAVALIER', 'source': 'bmf', 'display_name': 'ICO', 'display_section': 'bmf', 'format': 'str', 'format_cd': '', 'help': 'In Care Of', 'info_link': '', '': ''}
STREET {'original order': '4', 'key_name': 'STREET', 'sample': '15 ELLWOOD AVE', 'source': 'bmf', 'display_name': 'STREET', 'display_section': 'bmf', 'format': 'str', 'format_cd': '', 'help': 'Postal Address', 'info_link': '', '': ''}
CITY {'original order': '5', 'key_nam

In [ ]:
print (present_lu['EIN']['display_name'])
print (present_lu['EIN']['format'])
print (present_lu['EIN']['format_cd'])
print (present_lu['EIN']['help'])


EIN
str

Tax ID for organizations (get formal def)


In [ ]:
# get list of keys for each section
import pprint

sect_dict = {} # dict of list keys/fields in section

#
for s in  present_lu:
  sect_name = present_lu[s][2]
  print (sect_name, '  ', end='')
  #print (present_lu[s][0])
  print (s)
  if sect_name not in sect_dict:
    sect_dict[sect_name] = []

  sect_dict[sect_name].append(s)

# pprint.pprint(sect_dict)
print (sect_dict['Form 990x'])





display_section   key_name
bmf   EIN
bmf   NAME
bmf   ICO
bmf   STREET
bmf   CITY
bmf   STATE
bmf   ZIP
bmf   GROUP
bmf   SUBSECTION
bmf   AFFILIATION
bmf   CLASSIFICATION
bmf   RULING
bmf   DEDUCTIBILITY
bmf   FOUNDATION
bmf   ACTIVITY
bmf   ORGANIZATION
bmf   STATUS
bmf   TAX_PERIOD
bmf   ASSET_CD
bmf   INCOME_CD
bmf   FILING_REQ_CD
bmf   PF_FILING_REQ_CD
bmf   ACCT_PD
bmf   ASSET_AMT
bmf   INCOME_AMT
bmf   REVENUE_AMT
bmf   NTEE_CD
bmf   SORT_NAME
bmf   p_org_id
bmf   ntee_cat
bmf   ntee_define
bmf   CLASS1
bmf   CLASS2
bmf   CLASS3
bmf   CLASS4
bmf   ACT1
bmf   ACT2
bmf   ACT3
bmf   act1_lu
bmf   act2_lu
bmf   act3_lu
bmf   RULE_DT
bmf   aff_lu
bmf   org_lu
bmf   found_lu
Form 990x   filename
Form 990x   IRS_Form
Form 990x   TaxPeriodEndDt
Form 990x   TaxPeriodBeginDt
Form 990x   TaxYr
Form 990x   ReturnTypeCd
Form 990x   PhoneNum
Form 990x   BusinessNameLine1Txt
Form 990x   AddressLine1Txt
Form 990x   AddressLine2Txt
Form 990x   CityNm
Form 990x   StateAbbreviationCd
Form 990x   Z

# Tech Refs


## References on Pandas Merging

In [ ]:
# Handy References on Pandas Merge/Join, concat
# https://stackoverflow.com/questions/38256104/differences-between-merge-and-concat-in-pandas
# https://stackoverflow.com/questions/40468069/merge-two-dataframes-by-index/40468090#40468090
# https://pandas.pydata.org/docs/user_guide/merging.html
# https://stackoverflow.com/questions/53645882/pandas-merging-101

## check syntax streamlit app

In [ ]:
# filter based on index

display(np_local_df.filter(items=[2], axis=0))


,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,p_org_id,ntee_cat,ntee_define,CLASS1,CLASS2,CLASS3,CLASS4,ACT1,ACT2,ACT3,act1_lu,act2_lu,act3_lu,RULE_DT,aff_lu,org_lu,found_lu,filename,IRS_Form,TaxPeriodEndDt,TaxPeriodBeginDt,TaxYr,...,Total_Revenue,Mission,people,FormationYr,ActivityOrMissionDesc,TotalEmployeeCnt,MissionDesc,CYTotalRevenueAmt,Desc,status,coord_x,coord_y,cb_BASENAME,cb_BLKGRP,cb_BLOCK,cb_GEOID,cb_NAME,cb_TRACT,cb_COUNTY,cb_STATE,cnty_COUNTY,cnty_NAME,cnty_STATE,cong_BASENAME,cong_NAME,cntysub_NAME,cntysub_COUSUB,cntysub_GEOID,inc_PLACE,centracts_NAME,centracts_TRACT,csa_NAME,csa_GEOID,legup_NAME,leglow_NAME,num_matches,cntysub_PLACE,congress_rep_link,state_leg_lower_link,state_leg_upper_link,Unnamed: 0,url,found_name,snippet,full_result,ein,cluster_lng,cluster_lat,cluster_ind,cluster_ngroup
2,161557965,ACCESS TO INDEPENDENCE OF CORTLAND COUNTY INC,NaN,26 NORTH MAIN STREET,CORTLAND,NY,13045-2198,0,3,3,2000,199910,1,15,0,1,1,202309.0,5,5,1,0,9,534363.0,842554.0,842554.0,B80,NaN,43730,(B) Education,"Organizations that provide services, facilitie...",2,0,0,0,0,0,0,na,na,na,1999-10-01,Independent - This code is used if the organiz...,Corporation,Organization which receives a substantial part...,202441169349302154_public.xml,IRS990,2023-09-30,2022-10-01,2022.0,...,842554,(f990) THE MISSION OF ACCESS TO INDEPENDENCE I...,"[{'PersonNm': 'Alexandra Mikowski', 'TitleTxt'...",1998,THE MISSION OF ACCESS TO INDEPENDENCE IS TO EM...,16,THE MISSION OF ACCESS TO INDEPENDENCE IS TO EM...,842554,THE MISSION OF ACCESS TO INDEPENDENCE IS TO EM...,success,-76.180692,42.604318,3002.0,3.0,3002.0,3.602397e+14,Block 3002,970500.0,23.0,36.0,23.0,Cortland County,36.0,19.0,Congressional District 19,Cortland city,18388.0,3.602318e+09,18388.0,Census Tract 9705,970500.0,"Ithaca-Cortland, NY CSA",296.0,State Senate District 52,Assembly District 125,1.0,NaN,"<a href=""https://clerk.house.gov/members/M0012...","<a href=""https://www.assembly.state.ny.us/mem/...","<a href=""https://www.nysenate.gov/district/52""...",NaN,NaN,NaN,NaN,NaN,NaN,-76.181,42.604,1.0,68.0


# reports fiddle

In [ ]:
rpt_config = {
                  "rpt_name" :  { "rpt_title"  : "Human Readable Title",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            },

                  "alt1_rank" : { "rpt_title" : "Altair Test Rankings",
                            "rpt_def_name" : "not_implented",
                            "rpt_desc" :    "Rank of Number of Nonprofits and Rank of Total Income"
                            }
                }

print(some_dict['Moli'])
print(some_dict.keys())
print(some_dict.values())
print(some_dict.items())




rpt_list = list(rpt_config_old.values())
print (rpt_list)

#for rpt in rpt_config:
#    print (rpt_config[rpt]["rpt_title"])


reports_tmp = ["",
               "Cities, Nonprofit Income Rank",
                "Cities, Nonprofit Org Acount Rank",
                    "Cities, NP Income by Emphasis Area",
                    "Orgs and Income",
                    "Orgs and Affiliation",
                    'INCOME_CD', 'AFFILIATION', 'ORGANIZATION', 'FOUNDATION',
                    'SUBSECTION', 'ASSET_CD', 'DEDUCTIBILITY',
                    'alt_t1',
                    'alt1_rank', 'alt_city_pop_inc'
                    ]



[{'rpt_title': 'Human Readable Title', 'rpt_def_name': 'name_of_def', 'rpt_desc': 'Explanation of Report'}, {'rpt_title': 'Altair Test Rankings', 'rpt_def_name': 'not_implented', 'rpt_desc': 'Rank of Number of Nonprofits and Rank of Total Income'}]


In [ ]:
rpt_config = {
                  "rpt_name" :  { "rpt_title"  : "Human Readable Title",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            },
                  "alt1_rank" : { "rpt_title" : "Altair Test Rankings",
                            "rpt_def_name" : "not_implented",
                            "rpt_desc" :    "Rank of Number of Nonprofits and Rank of Total Income"
                            },
              "Rank Cities by NP Income" : { "rpt_title" :  "Rank Cities by NP Income",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            },
              "Rank Cities by NP Org Count" : { "rpt_title" : "NP Org Count Rank",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            },
              "NP Income by Emphasis Area" : { "rpt_title" : "NP Income by Emphasis Area",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            },
              "Orgs and Income" : { "rpt_title" : "Orgs and Income",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            },
              "Orgs and Affiliation" : { "rpt_title" : "Orgs and Affiliation",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            },
              "INCOME_CD" : { "rpt_title" : "INCOME_CD",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            },
              "AFFILIATION" : { "rpt_title" : "AFFILIATION",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            },
                "ORGANIZATION" : { "rpt_title" : "ORGANIZATION",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            },
                "FOUNDATION" : { "rpt_title" : "FOUNDATION",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            },
                "SUBSECTION" : { "rpt_title" : "SUBSECTION",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            },
                "ASSET_CD" : { "rpt_title" : "ASSET_CD",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            },
            "DEDUCTIBILITY" : { "rpt_title" : "DEDUCTIBILITY",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            },
            "alt_t1" : { "rpt_title" :  "alt_t1",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            },
            "alt1_rank" : { "rpt_title" : "Altair Test Rankings",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            },
            "alt_city_pop_inc" : { "rpt_title" : "alt_city_pop_inc",
                            "rpt_def_name" : "name_of_def",
                            "rpt_desc" : "Explanation of Report"
                            }
          }



rpt_names_list = list(rpt_config.keys())

my_list = [" "] + rpt_names_list
print (my_list)

print (rpt_config['alt1_rank']['rpt_title'])
print (rpt_config['alt1_rank']['rpt_desc'])




[' ', 'rpt_name', 'alt1_rank', 'Rank Cities by NP Income', 'Rank Cities by NP Org Count', 'NP Income by Emphasis Area', 'Orgs and Income', 'Orgs and Affiliation', 'INCOME_CD', 'AFFILIATION', 'ORGANIZATION', 'FOUNDATION', 'SUBSECTION', 'ASSET_CD', 'DEDUCTIBILITY', 'alt_t1', 'alt_city_pop_inc']
Altair Test Rankings
Explanation of Report
